In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import urllib
import base64

from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# Database connection constants.

username = "aacuser"
password = urllib.parse.quote("S3cretP@sswo4d")
DATABASE = "AAC"
COLLECTION = "animals"
HOST = "localhost"
PORT = 51859
shelter = AnimalShelter(HOST, PORT, DATABASE, COLLECTION, DATABASE, username, password)



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

# Define queries for various filters.
MOUNTAIN_WILDERNESS = {"animal_type": "Dog",
                 "breed": {"$in": ['German Shepherd', 'Alaskan Malamute'
                                   'Old English SHeepdog', 'Siberian Husky',
                                   'Rottweiler']},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26},
                 "age_upon_outcome_in_weeks": {"$lte": 156}}

WATER_RESCUE = {"animal_type": "Dog", "breed": {"$in": ['Labrador Retriever Mix', 
                                                         'Chesapeake Bay Retriever',
                                                         'Newfoundland']},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte":26},
                 "age_upon_outcome_in_weeks": {"$lte": 156}}

DISASTER_TRACKING = {"animal_type": "Dog",
                    "breed": {"$in": ['Doberman Pinsch',
                                     'German Shepherd',
                                     'Golden Retriever',
                                     'Bloodhound',
                                     'Rottweiler']},
                    "sex_upon_outcome": "Intact Male",
                    "age_upon_outcome_in_weeks": {"$gte": 20},
                    "age_upon_outcome_in_weeks": {"$lte": 300}}



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        # Include branding image and unique line to identify the developer
        html.A(href="https://www.snhu.edu", children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': "300px", 'width': '300px'})]),
        html.H2("Charles Haines - 2022 - SNHU CS340"),
        
    # Code for dropdown menu used to select the current filter
    dcc.Dropdown(id='filter-type', options=[{'label': "Show All", "value": "all"},
                                            {'label': "Water Rescue", 'value': "water_rescue"},
                                            {'label': "Mountain or Wilderness", 'value': "mountain_wilderness"},
                                            {'label': 'Disaster or Individual Tracking', 'value': 'disaster_tracking'}],
                value="all")

    ]),
    html.Hr(),
    
    # Data table 
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # Enable pagination and sorting options
        editable = False,
        sort_action = "native",
        sort_mode = "multi",
        row_deletable = False,
        page_action = "native",
        page_current = 0,
        page_size = 10,
        row_selectable = "single",
        selected_columns = [],
        
    ),
    html.Br(),
    html.Hr(),
    
    # Display chart and map
    html.Div(style={"text-align": "center"}, children=[
        html.H2('Animal Breed Graph and Location'),
        html.Br()
    ]),
    
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={"height": "600px"},
            children = [
                dcc.Graph(id="pie-chart-id")
            ]

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={"height": "600px"}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback function is used to update the dashboard based on the current filter selection.
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):

        # Filter the data using query constants defined at the top of this file.
        if filter_type == "water_rescue":
            df = pd.DataFrame.from_records(shelter.read(WATER_RESCUE))
        elif filter_type == "mountain_wilderness":
            df = pd.DataFrame.from_records(shelter.read(MOUNTAIN_WILDERNESS))
        elif filter_type == "disaster_tracking":
            df = pd.DataFrame.from_records(shelter.read(DISASTER_TRACKING))
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
        
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        
        data=df.to_dict('records')
        
        
        return (data,columns)
# This callback is used to highlight a cell if it is clicked on
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback updates the graph and is executed each time the filter option is changed.
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('filter-type', 'value')])
def update_graphs(viewData, value):
    viewDF = pd.DataFrame.from_dict(viewData)

    # Use pandas to create a new data frame by counting the number of each animal breed currently
    # being displayed.
    distincBreedsCount = viewDF['breed'].value_counts()
    df_result = pd.DataFrame(distincBreedsCount)
    df_result = df_result.reset_index()
    df_result.columns = ['Breed', 'Count']

    return dcc.Graph(figure=px.pie(df_result, values='Count', names='Breed', title="Animal Breed", template="plotly_dark"),
                    style={"height": "600px"})

# This callback is used to update the map and center on the location of the currently
# selected animal. If no animal is selected it will center on the location of the 
# animal in the top row of the table currently being displayed.
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row):
    # Create dataframe of the view and isolate the data of the selected row
    viewDF = pd.DataFrame.from_dict(viewData)
    
    # Display first row of the current viewport if no visable row selected
    if not row or row is None:
        row = [0]
    
    selectedAnimal = viewDF.iloc[row[0]]
    
    
    # Display a map using the coordinates from the animal's data
    coords = [selectedAnimal['location_lat'], selectedAnimal['location_long']]
    
    return html.Div(style={'height': '600px', 'width': '800px',
                           "margin": "auto"}, children = [
        dl.Map(center = coords,
              zoom = 15,
              children = [
                  dl.TileLayer(id="base-layer-id"),
                  dl.Marker(position = coords,
                           children = [
                               dl.Tooltip(selectedAnimal['breed']),
                               dl.Popup([
                                   html.H4("Animal Name"),
                                   html.P(selectedAnimal['name'])])
                            
                 ])
        ])
    ])

app